In [1]:
import numpy as np

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

In [3]:
#import data
dataset = pd.read_csv("C:/Users/FUGE/Desktop/HOLMUSK PROJECT/finalml.csv")

In [4]:
data = dataset.drop(['Unnamed: 0','patient_id'],axis = 1)

In [5]:
data.columns

Index(['totalamt', 'medical_history_1', 'medical_history_2',
       'medical_history_3', 'medical_history_4', 'medical_history_5',
       'medical_history_6', 'medical_history_7', 'preop_medication_1',
       'preop_medication_2', 'preop_medication_3', 'preop_medication_4',
       'preop_medication_5', 'preop_medication_6', 'symptom_1', 'symptom_2',
       'symptom_3', 'symptom_4', 'symptom_5', 'lab_result_1', 'lab_result_2',
       'lab_result_3', 'weight', 'height', 'age', 'stay', 'bmi', 'gender_F',
       'gender_M', 'race_Chinese', 'race_Indian', 'race_Malay', 'race_Others',
       'resident_status_Foreigner', 'resident_status_PR',
       'resident_status_Singaporean'],
      dtype='object')

In [6]:
colmean = data.loc[:,['lab_result_1', 'lab_result_2',
       'lab_result_3', 'weight', 'height', 'age', 'stay', 'bmi']].mean()
colstd = data.loc[:,['lab_result_1', 'lab_result_2',
       'lab_result_3', 'weight', 'height', 'age', 'stay', 'bmi']].std()

In [7]:
data.loc[:,['lab_result_1', 'lab_result_2',
       'lab_result_3', 'weight', 'height', 'age', 'stay', 'bmi']]= (data.loc[:,['lab_result_1', 'lab_result_2',
       'lab_result_3', 'weight', 'height', 'age', 'stay', 'bmi']]-colmean)/colstd  #data standardization

In [8]:
X = data.drop(['totalamt'],axis=1)
y = data.totalamt
y = (y - y.mean())/y.std()

In [10]:
y.head()

0   -1.641354
1    0.073107
2   -0.434441
3   -0.647265
4    5.082923
Name: totalamt, dtype: float64

In [11]:
from sklearn.cross_validation import train_test_split #split for training set and test set

In [66]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4) # 60% for training

In [67]:
X_cval, X_test, y_cval, y_test = train_test_split(X_val,y_val,test_size=0.5) #20% for CV,20% for test

In [68]:
X_train.shape

(2040, 35)

In [69]:
X_cval.shape

(680, 35)

In [70]:
X_test.shape

(680, 35)

In [71]:
import xgboost as xgb

In [73]:
X_train = np.asarray(X_train)

In [74]:
y_train = np.asarray(y_train)

In [102]:
X_test = X_test.values
y_test = y_test.values

In [107]:
X_cval = np.asarray(X_cval)

In [108]:
y_cval = np.asarray(y_cval)

In [58]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [113]:
# grid search
model = XGBRegressor()
param_grid = {'n_estimators':[1000],
              'max_depth':[3],
              'learning_rate':[0.1],
             'reg_lambda':[0],
             }
grid_search = GridSearchCV(model,param_grid,scoring='neg_mean_squared_error',cv=5)
grid_result = grid_search.fit(X_train,y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.025281 using {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'reg_lambda': 0}
-0.025281 (0.014964) with: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'reg_lambda': 0}


In [51]:
e = list(range(50,400,50))
dict(e = e)

{'e': [50, 100, 150, 200, 250, 300, 350]}

In [43]:
type(range(50,400,50))

range

In [110]:
clf = XGBRegressor(max_depth = 3,n_estimators = 1000,learning_rate =0.1,reg_lambda = 0)
clf.fit(X_train,y_train)
y_pfinal = clf.predict(X_cval)

In [111]:
yf = y_pfinal*10155+21859
yt = y_cval*10155+21859
xax = np.mean(abs(yf-yt))

In [112]:
err =abs(yf-yt)
error = pd.DataFrame(err,columns = ['error abs'])
error.describe()

,error abs
count,680.000000
mean,885.773550
std,1558.276552
min,0.797469
25%,233.741154
50%,501.994283
75%,950.976195
max,21096.365947


In [114]:
from sklearn.svm import SVR

In [137]:
# grid search  ##found poly kernel degree = 2 with better performance than SVR() --> 
##too few features need to add more polynominal ? should also try add polynominal in other model
model = SVR()
param_grid = {'C':[50,100,200,500],
             'kernel':['poly'],
             'degree':[2]}
grid_search = GridSearchCV(model,param_grid,scoring='neg_mean_squared_error',cv=5)
grid_result = grid_search.fit(X_train,y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.014207 using {'C': 100, 'degree': 2, 'kernel': 'poly'}
-0.015695 (0.008595) with: {'C': 50, 'degree': 2, 'kernel': 'poly'}
-0.014207 (0.006718) with: {'C': 100, 'degree': 2, 'kernel': 'poly'}
-0.014645 (0.006456) with: {'C': 200, 'degree': 2, 'kernel': 'poly'}
-0.014644 (0.006451) with: {'C': 500, 'degree': 2, 'kernel': 'poly'}


In [122]:
from sklearn import linear_model

In [129]:
# grid search LASSO
model = linear_model.Lasso()
param_grid = {'alpha':[0.0001,0.0003,0.0007,0.001,0.003]}
grid_search = GridSearchCV(model,param_grid,scoring='neg_mean_squared_error',cv=5)
grid_result = grid_search.fit(X_train,y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.069071 using {'alpha': 0.001}
-0.069248 (0.031796) with: {'alpha': 0.0001}
-0.069163 (0.031915) with: {'alpha': 0.0003}
-0.069073 (0.032164) with: {'alpha': 0.0007}
-0.069071 (0.032365) with: {'alpha': 0.001}
-0.070066 (0.034111) with: {'alpha': 0.003}


In [135]:
clf = SVR(C=100,kernel = 'poly',degree =2)
clf.fit(X_train,y_train)
y_pfinal = clf.predict(X_cval)

In [136]:
yf = y_pfinal*10155+21859
yt = y_cval*10155+21859
xax = np.mean(abs(yf-yt))
err =abs(yf-yt)
error = pd.DataFrame(err,columns = ['error abs'])
error.describe()

,error abs
count,680.000000
mean,693.956573
std,712.477372
min,1.502875
25%,242.194541
50%,551.845512
75%,919.307795
max,7016.002666


In [143]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [149]:
# grid search for RIDGE, and introduce some polynomial features using sklearn pipeline
model = make_pipeline(PolynomialFeatures(2), Ridge())
param_grid = {'polynomialfeatures__degree':[2,3,4,5],
    'ridge__alpha':[0.1,0.3,1,3]}
grid_search = GridSearchCV(model,param_grid,scoring='neg_mean_squared_error',cv=5)
grid_result = grid_search.fit(X_train,y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.005298 using {'polynomialfeatures__degree': 2, 'ridge__alpha': 0.3}
-0.005417 (0.002229) with: {'polynomialfeatures__degree': 2, 'ridge__alpha': 0.1}
-0.005298 (0.002528) with: {'polynomialfeatures__degree': 2, 'ridge__alpha': 0.3}
-0.005477 (0.003459) with: {'polynomialfeatures__degree': 2, 'ridge__alpha': 1}
-0.007188 (0.005526) with: {'polynomialfeatures__degree': 2, 'ridge__alpha': 3}
-0.030830 (0.014799) with: {'polynomialfeatures__degree': 3, 'ridge__alpha': 0.1}
-0.030854 (0.014815) with: {'polynomialfeatures__degree': 3, 'ridge__alpha': 0.3}
-0.030944 (0.014871) with: {'polynomialfeatures__degree': 3, 'ridge__alpha': 1}
-0.031233 (0.015031) with: {'polynomialfeatures__degree': 3, 'ridge__alpha': 3}
-0.074202 (0.030008) with: {'polynomialfeatures__degree': 4, 'ridge__alpha': 0.1}
-0.074203 (0.030009) with: {'polynomialfeatures__degree': 4, 'ridge__alpha': 0.3}
-0.074205 (0.030012) with: {'polynomialfeatures__degree': 4, 'ridge__alpha': 1}
-0.074213 (0.030023) with: {'po

In [147]:
model.get_params().keys() #to check the available parameters in param_grid

dict_keys(['steps', 'polynomialfeatures', 'ridge', 'polynomialfeatures__degree', 'polynomialfeatures__include_bias', 'polynomialfeatures__interaction_only', 'ridge__alpha', 'ridge__copy_X', 'ridge__fit_intercept', 'ridge__max_iter', 'ridge__normalize', 'ridge__random_state', 'ridge__solver', 'ridge__tol'])

In [158]:
#what if combine xgboost with polynomialfeatures? ----> even better ! (SEEMS OVER FITTING WHEN USE IT AGAIN)
model = make_pipeline(PolynomialFeatures(), XGBRegressor())
param_grid = {'polynomialfeatures__degree':[2],
              'xgbregressor__max_depth':[3],
              'xgbregressor__n_estimators':[1000],
              'xgbregressor__reg_lambda':[0.1,0.3,1,3]}
grid_search = GridSearchCV(model,param_grid,scoring='neg_mean_squared_error',cv=10)
grid_result = grid_search.fit(X_train,y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.028120 using {'polynomialfeatures__degree': 2, 'xgbregressor__max_depth': 3, 'xgbregressor__n_estimators': 1000, 'xgbregressor__reg_lambda': 3}
-0.029190 (0.019506) with: {'polynomialfeatures__degree': 2, 'xgbregressor__max_depth': 3, 'xgbregressor__n_estimators': 1000, 'xgbregressor__reg_lambda': 0.1}
-0.028418 (0.020150) with: {'polynomialfeatures__degree': 2, 'xgbregressor__max_depth': 3, 'xgbregressor__n_estimators': 1000, 'xgbregressor__reg_lambda': 0.3}
-0.029871 (0.021585) with: {'polynomialfeatures__degree': 2, 'xgbregressor__max_depth': 3, 'xgbregressor__n_estimators': 1000, 'xgbregressor__reg_lambda': 1}
-0.028120 (0.015580) with: {'polynomialfeatures__degree': 2, 'xgbregressor__max_depth': 3, 'xgbregressor__n_estimators': 1000, 'xgbregressor__reg_lambda': 3}


In [152]:
model.get_params().keys()

dict_keys(['steps', 'polynomialfeatures', 'xgbregressor', 'polynomialfeatures__degree', 'polynomialfeatures__include_bias', 'polynomialfeatures__interaction_only', 'xgbregressor__base_score', 'xgbregressor__colsample_bylevel', 'xgbregressor__colsample_bytree', 'xgbregressor__gamma', 'xgbregressor__learning_rate', 'xgbregressor__max_delta_step', 'xgbregressor__max_depth', 'xgbregressor__min_child_weight', 'xgbregressor__missing', 'xgbregressor__n_estimators', 'xgbregressor__nthread', 'xgbregressor__objective', 'xgbregressor__reg_alpha', 'xgbregressor__reg_lambda', 'xgbregressor__scale_pos_weight', 'xgbregressor__seed', 'xgbregressor__silent', 'xgbregressor__subsample'])